In [1]:
import sys 
import numpy as np 
import cv2


In [3]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("Image load failed!")
    sys.exit()
# sobel filter 
dx = cv2.Sobel(src, -1, 1, 0, delta=128)
dy = cv2.Sobel(src, -1, 0, 1, delta=128)

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.waitKey()

cv2.destroyAllWindows()

In [4]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("Image load failed!")
    sys.exit()
# scharr filter 

dx = cv2.Sobel(src, -1, 1, 0, delta=128)
dy = cv2.Sobel(src, -1, 0, 1, delta=128)

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.waitKey()

cv2.destroyAllWindows()




In [5]:
# canny edge 검출 
# 1. 가우시안 블러링 why? 노이즈 제거
# 2. gradient (크기 및 방향 계산) why? edge 방향을 알기 위해
# 3. 비최대 억제 why? edge가 아닌 부분을 제거
# 4. 이중 임계값을 이용한 히스테리시스 why? edge인지 아닌지 판단하기 위해

src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("Image load failed!")
    sys.exit()
# scharr filter 

dst = cv2.Canny(src, 50, 150)
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()




In [ ]:
import sys
import cv2
import numpy as np 
# 이미지 읽기
src = cv2.imread('./data/dial.jpg') 
if src is None:
    print("Image load failed!")
    sys.exit()

# 그레이스케일로 변환 (HoughCircles는 그레이스케일 이미지만 가능)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# GaussianBlur로 노이즈 제거
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)

def on_trackbar(pos):
    rmin = cv2.getTrackbarPos('minRadius', 'img')
    rmax = cv2.getTrackbarPos('maxRadius', 'img')
    th = cv2.getTrackbarPos('threshold', 'img')

    # HoughCircles 함수 호출
    circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50,
        param1=120, param2=th, minRadius=rmin, maxRadius=rmax)

    dst = src.copy()
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")  # 좌표와 반지름을 정수로 변환
        for i in range(circles.shape[0]):
            cx, cy, radius = circles[i]
            cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('img', dst)

# 트랙바 생성
cv2.imshow('img', src)
cv2.createTrackbar('minRadius', 'img', 0, 100, on_trackbar)
cv2.createTrackbar('maxRadius', 'img', 0, 150, on_trackbar)
cv2.createTrackbar('threshold', 'img', 0, 100, on_trackbar)
cv2.setTrackbarPos('minRadius', 'img', 10)
cv2.setTrackbarPos('maxRadius', 'img', 80)
cv2.setTrackbarPos('threshold', 'img', 40)

cv2.waitKey()

cv2.destroyAllWindows()


In [ ]:
import sys
import numpy as np
import cv2
# 입력 이미지 불러오기
src = cv2.imread('./data/coins1.jpg')
if src is None:
    print('Image open failed!')
    sys.exit()


gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1)

# 허프 변환 원 검출
circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50,
    param1=150, param2=40, minRadius=20, maxRadius=80)
# 원 검출 결과 및 동전 금액 출력
sum_of_money = 0
dst = src.copy()

if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = circles[0][i]
        # Convert cx, cy, and radius to integers if they are not already
        cx, cy, radius = int(cx), int(cy), int(radius)
        cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)
        # 동전 영역 부분 영상 추출
        x1 = int(cx - radius)
        y1 = int(cy - radius)
        x2 = int(cx + radius)
        y2 = int(cy + radius)
        radius = int(radius)

        crop = dst[y1:y2, x1:x2, :]
        ch, cw = crop.shape[:2]
        
        # 동전 영역에 대한 ROI 마스크 영상 생성
        mask = np.zeros((ch, cw), np.uint8)
        cv2.circle(mask, (cw//2, ch//2), radius, 255, -1)
        # 동전 영역 Hue 색 성분을 +40 시프트하고, Hue 평균을 계산
        hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        hue, _, _ = cv2.split(hsv)
        hue_shift = (hue + 40) % 180
        mean_of_hue = cv2.mean(hue_shift, mask)[0]
        print(mean_of_hue)
        # Hue 평균이 53보다 작으면 10원, 90보다 크면 100원으로 간주
        won = 100
        if mean_of_hue < 53:
            won = 10
        sum_of_money += won
        cv2.putText(crop, str(won), (20, 50), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (255, 0, 0), 2, cv2.LINE_AA)

cv2.putText(dst, str(sum_of_money) + ' won', (40, 80),
    cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

61.007379494426125
61.873783798312104
54.942120051085574
57.85006385696041
50.11636270228615
50.6781252768672
51.99583135954524
48.027953441481074
